### 품사 태깅

In [18]:
# csv파일 불러옴
import csv
import re

raw = []
f = open(r'C:\Users\thehb\OneDrive\Desktop\sentenceExtraction_with_Article\ytn_article.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)
for line in rdr:
    raw.append(line)
f.close()  

# 빈 text 제거 & 특수문자 제거
text = []
for i in range(len(raw)):
    if(raw[i][0] is not ' '):
        newtext = re.sub('[-=+,#/\?:^$@*\"~&%!<>()"]', '', raw[i][0])
        text.append(newtext)

In [19]:
# 품사 태깅(주어진 텍스트를 형태소 단위로 나누고 나눠진 형태소를 해당하는 품사와 함께 리스트화)
# stemming(단어들을 원형으로 포현), normalization(표현 방법이 다른 단어들을 통합시켜 같은 단어로 만듬)
import json
import os
from konlpy.tag import Okt

okt = Okt()

def tokenize(doc): # 형태소 분석기인 Okt를 이용해서 품사를 태깅시킴
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

os.chdir(r'C:\Users\thehb\OneDrive\Desktop\sentenceExtraction_with_Article') # 경로 설정

In [20]:
if os.path.isfile('text_docs.json'): # 경로에 text_docs.json 파일이 존재하면 json파일을 만들지 않고 load함
    with open('text_docs.json', encoding='utf-8') as f:
        text_docs = json.load(f)
else: # json파일이 없을 경우 text를 품사 태깅시킨 데이터로 변환시켜 json파일로 저장
    text_docs = [(tokenize(row)) for row in text]
    # JSON 파일로 저장
    with open('text_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(text_docs, make_file, ensure_ascii=False, indent="\t")

In [21]:
# 태깅된 각 형태소들을 하나의 list로 합침
tokens = [t for d in text_docs for t in d]
print(len(tokens))

1707035


In [22]:
import nltk
from pprint import pprint

nltk_text = nltk.Text(tokens, name='NMSC')

# 전체 토큰의 개수
total_tokens = nltk_text.tokens
print(len(total_tokens))

# 중복을 제외한 토큰의 개수
set_total_tokens = list(set(nltk_text.tokens))
print(len(set_total_tokens))            

# 출현 빈도가 높은 상위 토큰 10개
#pprint(nltk_text.vocab().most_common(50))

1707035
24065


### Word2vec

In [23]:
# 빈도수가 50개 이상인 token 확인.
most_common_tokens = nltk_text.vocab().most_common(len(set_total_tokens))
for i in range(len(set_total_tokens)):
    if(most_common_tokens[i][1]<50):
        index = i
        break
most_common_tokens_50 = most_common_tokens[:index]

In [26]:
# Word embedding(Word2vec 사용)
from gensim.models import Word2Vec
embedding_model = Word2Vec(text_docs, size=100, window = 2, min_count=50, workers=4, iter=100, sg=1)

C:\Users\thehb\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [58]:
embedding_model.save(r'C:\Users\thehb\Code\sentenceExtraction_with_Article\embedding_model') # 모델 저장

In [27]:
for s in set_total_tokens:
    if("피해" in s):
        print(s)

피해망상/Noun
피해자/Noun
피해/Noun


In [28]:
# 두 벡터 사이의 코사인 유사도를 구함. 값이 클수록 비슷한 단어
word_similar = embedding_model.most_similar(positive=["피해/Noun"], topn=10)
#print(embedding_model.most_similar(positive=["피해/Noun"], topn=100))
print([i[0] for i in word_similar])

['인명/Noun', '침수/Noun', '규모/Noun', '재산/Noun', '화재/Noun', '농작물/Noun', '농가/Noun', '손해/Noun', '대비/Noun', '폐/Noun']


C:\Users\thehb\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [29]:
words_damage = ['인명', '침수', '재산', '화재', '사고', '농작물', '사망자']

## 기사를 문장별로 토큰화

In [30]:
from konlpy.tag import Kkma
kkma = Kkma()

In [32]:
sentence_tokens = [] # 기사를 문장별로 토큰화한 결과값을 list에 저장
articles = list(set(text)) # 중복되는 기사 제거
for article in articles:
    sentence_tokens.append(kkma.sentences(article))    

## '피해'라는 단어와 top7 단어가 함께 들어있는 문장 추출

In [33]:
sentences_damage = []
for sentences in sentence_tokens:
    for sentence in sentences:
        isExist = False        
        for word_damage in words_damage:
            if isExist == False:
                if (word_damage in sentence) and ('피해' in sentence):
                    sentences_damage.append(sentence)
                    isExist = True

In [34]:
sentences_damage[10:20]

['따라서 침수 피해 없도록 배수로 점검해야 되겠고 특히 밤에 제주도 통과하기 때문에 대피 준비도 우리가 말하는 손전등 요즘은 다 휴대폰들 가지고 다니니까 통신 문제는 그렇게 큰 문제가 없는데. 문제는 많은 비로 강풍으로 인해서 정전이 되는 사태가 발생을 하거든요.',
 '산사태와 침수 피해가 우려되는 만큼 대비 철저히 해 주시기 바랍니다.',
 '잇따른 농작물 피해에 이어 계곡과 약수터의 물까지 말라가고 있습니다.',
 '한국도로 공사는 날씨로 인해 도로 상황이 좋았고 최근 대형 교통사고가 여러 건 발생하면서 운전자들의 경각심이 높아 진 것도 사고 피해가 감소한 원인으로 보인다고 설명했습니다.',
 '신 현우 옥시 전 사장 등 증인과 참고인 18명이 출석하지 않기로 하거나 출석 여부가 불확실한 가운데 특위는 정부와 업체 관계자들을 상대로 가습기 살균제 피해 사고의 책임 소재와 재발방지 대책을 추궁하고 있습니다.',
 '시설물 피해와 안전사고가 우려되는 상황입니다.',
 '물놀이를 할 때는 안전관리요원이 있는 곳에서 구명조끼 등 안전장비를 착용해야 인명 피해를 줄일 수 있습니다.',
 '[ 앵커] 어제 충북과 강원 경북 등 중부 지역 곳곳에 굵은 우박이 떨어졌는데요 짧은 시간 쏟아졌지만 추석을 앞두고 사과 등 농작물이 큰 피해를 봤습니다.',
 '우박은 국지적이며 돌발적인 현상으로 정확한 발생지역을 사실상 예측하기가 어렵고 얼음 알갱이가 떨어지는 힘 때문에 짧은 시간의 우 박에도 농작물에 심각한 피해를 주게 됩니다.',
 '그런 데 이번 우박으로 농작물 피해가 클 것 같은데 상황 어떤 가요[ 기자] 네']

In [35]:
len(sentences_damage)

369

### 추출된 문장 csv파일로 저장

In [36]:
f = open(r'C:\Users\thehb\Code\sentenceExtraction_with_Article\sentences_damage.csv', 'w', encoding='utf-8', newline='')
wr = csv.writer(f)
for i in sentences_damage:
    wr.writerow([i])
f.close()